In [77]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from numba import jit
from sklearn.linear_model import LinearRegression  # for linear regression
from sklearn.tree import DecisionTreeClassifier  # for decision tree mining
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [80]:
file_export = 'export2012.csv'
data = pd.read_csv(file_export)
df_sorted= data.sort_values(by=['case', 'startTime'])
data = df_sorted.copy().loc[:,['case', 'event', 'startTime', 'completeTime']]

In [81]:
data = pd.read_csv(file_export)

In [82]:
data_sorted = data.sort_values(by=['case', 'event','completeTime'])

In [83]:
data_iterrated = data.copy()

@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]
        idx+=1
    return res

data_iterrated["next_task"] = calculator_nb(data_iterrated['case'].values, data_iterrated["event"].values)

C:\Users\chbak\AppData\Local\Temp/ipykernel_13132/687891360.py:9: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13132\687891360.py", line 11:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
numba.core.transforms INFO  finding looplift candidates
C:\Users\chbak\AppData\Local\Temp/ipykernel_13132/687891360.py:9: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13132\687891360.py", line 11:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "

In [84]:
data_train, data_test = train_test_split(data_iterrated,test_size=0.2, shuffle=False)

In [85]:
data_iterrated.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED


In [86]:
list_of_events = data_iterrated["event"].unique()
event_to_num = {}
i=0
for event in list_of_events:
    event_to_num[str(event)] = i
    i += 1

In [87]:
event_to_num['None'] = 23

Naive predictor. First, event predictor is constructed.

In [88]:
#For each event, counts how many times another event occurs after it.
#Indexed based on list_of_events
count = {}
# initialise the count dictionary with zeros, otherwise it does not work
for event in list_of_events:
        count[str(event)] = [0 for i in range(len(list_of_events))]
 

for event in list_of_events:
    for k in range(len(list_of_events)):
        count[str(event)][k] = data_train[ (data_train['event'] == event) & (data_train['next_task'] == list_of_events[k]) ].count()['event']
            
count

{'A_SUBMITTED': [0,
  10291,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'A_PARTLYSUBMITTED': [0,
  3886,
  3757,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2648,
  0,
  3832,
  0,
  0,
  54],
 'A_PREACCEPTED': [0,
  0,
  5776,
  5776,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'W_Completeren aanvraag': [0,
  0,
  0,
  12770,
  4017,
  22,
  12,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  869,
  908,
  0,
  0,
  0,
  3],
 'A_ACCEPTED': [0,
  0,
  0,
  53,
  0,
  2295,
  1615,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  19,
  35,
  0,
  0,
  0,
  0],
 'O_SELECTED': [0,
  0,
  0,
  0,
  0,
  1,
  2317,
  2463,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  789,
  0,
  0,
  0,
  0,
  0,
  0],
 'A_FINALIZED': [0,
  0,
  0,
  0,
  0,
  1627,
  0,
  2317,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'O_CRE

In [89]:
#The main part of the naive predictor
#Takes the above dictionary and finds the most frequenly occuring consecutive
#event after each event
#Basically, it is our trained model that can be mapped to the original dataset
count2 = count
index = {}
for event in list_of_events:
    max = 0
    for k in range( len(list_of_events)-1 ):
        if count2[event][k] > max:
            max = count2[event][k]
            #print(max)
            index2 = k
    index[event] = index2
index

{'A_SUBMITTED': 1,
 'A_PARTLYSUBMITTED': 1,
 'A_PREACCEPTED': 2,
 'W_Completeren aanvraag': 3,
 'A_ACCEPTED': 5,
 'O_SELECTED': 7,
 'A_FINALIZED': 7,
 'O_CREATED': 8,
 'O_SENT': 8,
 'W_Nabellen offertes': 9,
 'O_SENT_BACK': 10,
 'W_Valideren aanvraag': 11,
 'A_REGISTERED': 15,
 'A_APPROVED': 15,
 'O_ACCEPTED': 12,
 'A_ACTIVATED': 12,
 'O_CANCELLED': 5,
 'A_DECLINED': 20,
 'A_CANCELLED': 16,
 'W_Afhandelen leads': 2,
 'O_DECLINED': 17,
 'W_Nabellen incomplete dossiers': 21,
 'W_Beoordelen fraude': 17}

In [90]:
#Turns the above into an event to event dictionary 

prediction = {}
for event in list_of_events:
    prediction[event] = list_of_events[index[event]]
    
prediction

{'A_SUBMITTED': 'A_PARTLYSUBMITTED',
 'A_PARTLYSUBMITTED': 'A_PARTLYSUBMITTED',
 'A_PREACCEPTED': 'A_PREACCEPTED',
 'W_Completeren aanvraag': 'W_Completeren aanvraag',
 'A_ACCEPTED': 'O_SELECTED',
 'O_SELECTED': 'O_CREATED',
 'A_FINALIZED': 'O_CREATED',
 'O_CREATED': 'O_SENT',
 'O_SENT': 'O_SENT',
 'W_Nabellen offertes': 'W_Nabellen offertes',
 'O_SENT_BACK': 'O_SENT_BACK',
 'W_Valideren aanvraag': 'W_Valideren aanvraag',
 'A_REGISTERED': 'A_ACTIVATED',
 'A_APPROVED': 'A_ACTIVATED',
 'O_ACCEPTED': 'A_REGISTERED',
 'A_ACTIVATED': 'A_REGISTERED',
 'O_CANCELLED': 'O_SELECTED',
 'A_DECLINED': 'O_DECLINED',
 'A_CANCELLED': 'O_CANCELLED',
 'W_Afhandelen leads': 'A_PREACCEPTED',
 'O_DECLINED': 'A_DECLINED',
 'W_Nabellen incomplete dossiers': 'W_Nabellen incomplete dossiers',
 'W_Beoordelen fraude': 'A_DECLINED'}

In [91]:
#Adds predicted event column
df_predicted_event = data_test.copy()
df_predicted_event["predicted_event"] = ["" for i in df_predicted_event["event"]]
df_predicted_event['predicted_event'] = df_predicted_event["event"].map(prediction)
df_predicted_event.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,predicted_event
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,O_DECLINED,O_DECLINED
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,None,A_DECLINED
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,A_PARTLYSUBMITTED
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,W_Afhandelen leads,A_PARTLYSUBMITTED
152666,205760,W_Afhandelen leads,2012/02/02 09:13:13.084,2012/02/02 09:22:33.272,8000,2012/02/01 21:51:59.169,11169,W_Afhandelen leads,A_PREACCEPTED
152667,205760,W_Afhandelen leads,2012/02/02 09:55:00.609,2012/02/02 09:59:16.870,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,A_PREACCEPTED
152668,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,A_PREACCEPTED
152669,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,W_Completeren aanvraag,A_PREACCEPTED
152670,205760,W_Completeren aanvraag,2012/02/02 11:37:56.472,2012/02/02 11:40:30.356,8000,2012/02/01 21:51:59.169,11201,W_Completeren aanvraag,W_Completeren aanvraag


In [92]:
df_new3 = df_predicted_event.copy()
df_new3["next_task"] = df_new3["next_task"].map(event_to_num)
df_new3["predicted_event"] = df_new3["predicted_event"].map(event_to_num)
df_new3.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,predicted_event
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,20.0,20
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,NaN,17
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,1.0,1
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,1.0,1
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,19.0,1


In [93]:
y_true = df_new3["next_task"].to_numpy().astype(int)
y_pred = df_new3["predicted_event"].to_numpy().astype(int)
y_true, y_pred

(array([         20, -2147483648,           1, ...,          19,
                 17, -2147483648]),
 array([20, 17,  1, ...,  1,  2, 20]))

In [94]:
f1_score(y_true, y_pred, average='micro')

0.5565424723575958

Naive time predictor

In [95]:
df_predicted_time = data_train.copy()
df_predicted_time["duration"] = pd.to_datetime(df_predicted_time["completeTime"]) - pd.to_datetime(df_predicted_time["startTime"])

#Sums up count for each event and the time each event takes
events_count = df_predicted_time.groupby("event")['duration'].agg('count')
event_duration_sum = df_predicted_time.groupby("event")['duration'].agg('sum')

#Computes average duration per event (basically our trained data that can be mapped onto test data)
duration_per_event = event_duration_sum / events_count 

In [96]:
#Final cleanup
df_predicted = df_predicted_event.copy()
df_predicted["predicted_duration"] = df_predicted['event'].map(duration_per_event)
df_predicted = df_predicted.drop(['next_task'], axis=1)
df_predicted.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,predicted_event,predicted_duration
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,O_DECLINED,0 days 00:00:00
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,A_DECLINED,0 days 00:00:00
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,0 days 00:00:00
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,0 days 00:00:00
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,0 days 00:00:00
152666,205760,W_Afhandelen leads,2012/02/02 09:13:13.084,2012/02/02 09:22:33.272,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:15:26.140431102
152667,205760,W_Afhandelen leads,2012/02/02 09:55:00.609,2012/02/02 09:59:16.870,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:15:26.140431102
152668,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:00:00
152669,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:00:00
152670,205760,W_Completeren aanvraag,2012/02/02 11:37:56.472,2012/02/02 11:40:30.356,8000,2012/02/01 21:51:59.169,11201,W_Completeren aanvraag,0 days 00:10:28.478056566


In [97]:
#Adding predicted start time of next event
df_predicted_final = df_predicted.copy()
df_predicted_final['predicted_time'] = df_predicted_final["predicted_duration"] + pd.to_datetime(df_predicted_final["startTime"])
df_predicted_final.head(10)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,predicted_event,predicted_duration,predicted_time
152661,205757,A_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,O_DECLINED,0 days 00:00:00,2012-02-20 12:11:35.191000000
152662,205757,O_DECLINED,2012/02/20 12:11:35.191,2012/02/20 12:11:35.191,15000,2012/02/01 21:44:38.752,10629,A_DECLINED,0 days 00:00:00,2012-02-20 12:11:35.191000000
152663,205760,A_SUBMITTED,2012/02/01 21:51:59.170,2012/02/01 21:51:59.170,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,0 days 00:00:00,2012-02-01 21:51:59.170000000
152664,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,0 days 00:00:00,2012-02-01 21:51:59.265000000
152665,205760,A_PARTLYSUBMITTED,2012/02/01 21:51:59.265,2012/02/01 21:51:59.265,8000,2012/02/01 21:51:59.169,112,A_PARTLYSUBMITTED,0 days 00:00:00,2012-02-01 21:51:59.265000000
152666,205760,W_Afhandelen leads,2012/02/02 09:13:13.084,2012/02/02 09:22:33.272,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:15:26.140431102,2012-02-02 09:28:39.224431102
152667,205760,W_Afhandelen leads,2012/02/02 09:55:00.609,2012/02/02 09:59:16.870,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:15:26.140431102,2012-02-02 10:10:26.749431102
152668,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:00:00,2012-02-02 09:59:15.677000000
152669,205760,A_PREACCEPTED,2012/02/02 09:59:15.677,2012/02/02 09:59:15.677,8000,2012/02/01 21:51:59.169,11169,A_PREACCEPTED,0 days 00:00:00,2012-02-02 09:59:15.677000000
152670,205760,W_Completeren aanvraag,2012/02/02 11:37:56.472,2012/02/02 11:40:30.356,8000,2012/02/01 21:51:59.169,11201,W_Completeren aanvraag,0 days 00:10:28.478056566,2012-02-02 11:48:24.950056566


Linear regression

In [98]:
data_iterrated_two = data_iterrated["event"].copy()
data_iterrated_two

0                    A_SUBMITTED
1              A_PARTLYSUBMITTED
2                  A_PREACCEPTED
3                  A_PREACCEPTED
4         W_Completeren aanvraag
                   ...          
190822               A_SUBMITTED
190823         A_PARTLYSUBMITTED
190824         A_PARTLYSUBMITTED
190825        W_Afhandelen leads
190826                A_DECLINED
Name: event, Length: 190827, dtype: object

In [99]:
for i in range(len(data_iterrated_two)):
    data_iterrated_two[i] = event_to_num[data_iterrated_two[i]]

In [100]:
event_to_num

{'A_SUBMITTED': 0,
 'A_PARTLYSUBMITTED': 1,
 'A_PREACCEPTED': 2,
 'W_Completeren aanvraag': 3,
 'A_ACCEPTED': 4,
 'O_SELECTED': 5,
 'A_FINALIZED': 6,
 'O_CREATED': 7,
 'O_SENT': 8,
 'W_Nabellen offertes': 9,
 'O_SENT_BACK': 10,
 'W_Valideren aanvraag': 11,
 'A_REGISTERED': 12,
 'A_APPROVED': 13,
 'O_ACCEPTED': 14,
 'A_ACTIVATED': 15,
 'O_CANCELLED': 16,
 'A_DECLINED': 17,
 'A_CANCELLED': 18,
 'W_Afhandelen leads': 19,
 'O_DECLINED': 20,
 'W_Nabellen incomplete dossiers': 21,
 'W_Beoordelen fraude': 22,
 'None': 23}

In [101]:
next_task = data_iterrated["next_task"].copy()
for i in range(len(next_task)):
    try:
        next_task[i] = event_to_num[next_task[i]]
    except:
        next_task[i] = 23

In [102]:
data_to_lr = data_iterrated.copy()
data_to_lr["event"] = data_iterrated_two
data_to_lr["next_task"] = next_task
data_to_lr

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task
0,173688,0,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,1
1,173688,1,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,2
2,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,2
3,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,3
4,173688,3,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,4
...,...,...,...,...,...,...,...,...
190822,214376,0,2012/02/29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,1
190823,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,1
190824,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,19
190825,214376,19,2012/03/01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,17


In [103]:
x = data_to_lr[["event"]]
y = data_to_lr["next_task"]

In [104]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
naive_predictor = LinearRegression()
naive_predictor.fit(X_train, y_train)
naive_predictor.score(X_test, y_test)
prediction = naive_predictor.predict(data_to_lr[["event"]]).round()

In [105]:
result_data = data_to_lr.copy()
result_data["prediction"] = prediction
result_data["prediction"] = result_data["prediction"].astype(int)

In [106]:
result_data

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction
0,173688,0,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,1,3
1,173688,1,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,2,4
2,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,2,5
3,173688,2,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,3,5
4,173688,3,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,4,6
...,...,...,...,...,...,...,...,...,...
190822,214376,0,2012/02/29 23:51:16.799,2012/02/29 23:51:16.799,15000,2012/02/29 23:51:16.799,112,1,3
190823,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,1,4
190824,214376,1,2012/02/29 23:51:17.423,2012/02/29 23:51:17.423,15000,2012/02/29 23:51:16.799,112,19,4
190825,214376,19,2012/03/01 09:26:46.736,2012/03/01 09:27:41.325,15000,2012/02/29 23:51:16.799,11169,17,18


In [107]:
num_to_event = {v: k for k, v in event_to_num.items()}
num_to_event

{0: 'A_SUBMITTED',
 1: 'A_PARTLYSUBMITTED',
 2: 'A_PREACCEPTED',
 3: 'W_Completeren aanvraag',
 4: 'A_ACCEPTED',
 5: 'O_SELECTED',
 6: 'A_FINALIZED',
 7: 'O_CREATED',
 8: 'O_SENT',
 9: 'W_Nabellen offertes',
 10: 'O_SENT_BACK',
 11: 'W_Valideren aanvraag',
 12: 'A_REGISTERED',
 13: 'A_APPROVED',
 14: 'O_ACCEPTED',
 15: 'A_ACTIVATED',
 16: 'O_CANCELLED',
 17: 'A_DECLINED',
 18: 'A_CANCELLED',
 19: 'W_Afhandelen leads',
 20: 'O_DECLINED',
 21: 'W_Nabellen incomplete dossiers',
 22: 'W_Beoordelen fraude',
 23: 'None'}

In [108]:
predicted_event_str = result_data["prediction"].copy()
for i in range(len(predicted_event_str)):
    predicted_event_str[i] = num_to_event[predicted_event_str[i]]

In [109]:
predicted_event_str

0         W_Completeren aanvraag
1                     A_ACCEPTED
2                     O_SELECTED
3                     O_SELECTED
4                    A_FINALIZED
                   ...          
190822    W_Completeren aanvraag
190823                A_ACCEPTED
190824                A_ACCEPTED
190825               A_CANCELLED
190826                A_DECLINED
Name: prediction, Length: 190827, dtype: object

In [110]:
result_data_str = data_iterrated.copy()
result_data_str["prediction"] = predicted_event_str

In [111]:
result_data_str["duration"] = pd.to_datetime(result_data_str["completeTime"]) - pd.to_datetime(result_data_str["startTime"])
result_data_str.head()

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction,duration
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED,W_Completeren aanvraag,0 days 00:00:00
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,A_ACCEPTED,0 days 00:00:00
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,O_SELECTED,0 days 00:00:00
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag,O_SELECTED,0 days 00:00:00
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED,A_FINALIZED,0 days 00:08:27.480000


In [112]:

events_count = result_data_str.groupby("event")['duration'].agg('count')
event_duration_sum = result_data_str.groupby("event")['duration'].agg('sum')

duration_per_event = event_duration_sum / events_count

@jit(parallel = True)
def make_prediction(event, dict):
    res = np.empty(len(event), dtype=object)
    for idx,_ in enumerate(event):    
        res[idx] = dict[event[idx]]
    return res

result_data_str["duration_prediction"] = make_prediction(result_data_str['event'].values, duration_per_event)

C:\Users\chbak\AppData\Local\Temp/ipykernel_13132/4069836337.py:6: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "make_prediction" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13132\4069836337.py", line 8:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
numba.core.transforms INFO  finding looplift candidates
C:\Users\chbak\AppData\Local\Temp/ipykernel_13132/4069836337.py:6: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "make_prediction" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13132\4069836337.py", line 8:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Func

In [113]:
result_data_str.head(5)

,case,event,startTime,completeTime,AMOUNT_REQ,REG_DATE,org:resource,next_task,prediction,duration,duration_prediction
0,173688,A_SUBMITTED,2011/10/01 00:38:44.546,2011/10/01 00:38:44.546,20000,2011/10/01 00:38:44.546,112,A_PARTLYSUBMITTED,W_Completeren aanvraag,0 days 00:00:00,0 days 00:00:00
1,173688,A_PARTLYSUBMITTED,2011/10/01 00:38:44.880,2011/10/01 00:38:44.880,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,A_ACCEPTED,0 days 00:00:00,0 days 00:00:00
2,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,A_PREACCEPTED,O_SELECTED,0 days 00:00:00,0 days 00:00:00
3,173688,A_PREACCEPTED,2011/10/01 00:39:37.906,2011/10/01 00:39:37.906,20000,2011/10/01 00:38:44.546,112,W_Completeren aanvraag,O_SELECTED,0 days 00:00:00,0 days 00:00:00
4,173688,W_Completeren aanvraag,2011/10/01 11:36:46.437,2011/10/01 11:45:13.917,20000,2011/10/01 00:38:44.546,112,A_ACCEPTED,A_FINALIZED,0 days 00:08:27.480000,0 days 00:10:00.791783229


More advanced algorithms (from Hristo)

In [115]:
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)][:100]

df_filtered=df_filtered.dropna() #Check later if dropna messes with the dataset through "None" tasks being interpreted as NaN
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])

In [116]:
#OLS regression
import statsmodels.api as sm
df_filtered = result_data[result_data["case"] < len(result_data)]
df_prediction = result_data[result_data["case"] >= len(result_data)]


df_filtered=df_filtered.dropna()
df_prediction=df_prediction.dropna()
df_filtered["startTime"] = pd.to_datetime(df_filtered["startTime"])
df_prediction["startTime"] = pd.to_datetime(df_prediction["startTime"])

y = np.array(pd.to_numeric(df_filtered["startTime"].values), dtype=float)
X = np.array(df_filtered["event"].values, dtype=float)
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
ypred = olsres.predict(X)
Xnew = np.array(df_prediction["event"].values, dtype=float)
ynewpred = olsres.predict(Xnew) 
df_prediction["time_OLS"] = pd.to_datetime(ynewpred)

In [131]:
# Lasso Regression
from sklearn import linear_model

y = np.floor(pd.to_numeric(df_filtered["startTime"].values), dtype=float)
X = df_filtered[["event", "case"]]

lassoReg = linear_model.Lasso()
lassoReg.fit(X, y)

Xnew = np.array(df_prediction[["event", "case"]].values, dtype=float)
df_prediction["time_Lasso"] = pd.to_datetime(lassoReg.predict(Xnew))

C:\Users\chbak\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.640e+34, tolerance: 2.582e+31
  model = cd_fast.enet_coordinate_descent(
C:\Users\chbak\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
C:\Users\chbak\AppData\Local\Temp/ipykernel_13132/1073336942.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prediction["time_Lasso"] = pd.to_datetime(lassoReg.predict(Xnew))


Random Fores and Decision Tree for event

In [118]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [119]:
# Prep data
result_data.sort_values(by=["case", "event"], inplace=True)
result_data_copy = result_data.copy()
result_data_copy['startTime'] = pd.to_numeric(pd.to_datetime(result_data_copy['startTime']))
result_data_copy['completeTime'] = pd.to_numeric(pd.to_datetime(result_data_copy['completeTime']))
result_data_copy['REG_DATE'] = pd.to_numeric(pd.to_datetime(result_data_copy['REG_DATE']))
result_data_copy.drop(columns='prediction', inplace=True)

In [120]:
# Split training and test and ensure floating point numbers
train_df, test_df = train_test_split(result_data_copy)

X_train = train_df.drop(['next_task'], axis=1)
Y_train = train_df['next_task']
X_train = X_train.values.astype("float32")
Y_train = Y_train.values.astype("float32")

X_test = test_df.drop(['next_task'], axis=1)
Y_test = test_df['next_task']
X_test = X_test.values.astype("float32")
Y_test = Y_test.values.astype("float32")

In [121]:
# Fitting and prediction 
DT = DecisionTreeClassifier()
RF = RandomForestClassifier()

DT_fit = DT.fit(X_train, Y_train)
RF_fit = RF.fit(X_train, Y_train)

DT_pred = DT_fit.predict(X_test)
RF_pred = RF_fit.predict(X_test)

In [122]:
# Accuracy
DT_pred = np.round(DT_pred)
RF_pred = np.round(RF_pred)
print("Decision Trees is %f percent accurate" % (accuracy_score(DT_pred, Y_test)*100))
print("Random Forests is %f percent accurate" % (accuracy_score(RF_pred, Y_test)*100))

Decision Trees is 48.684679 percent accurate
Random Forests is 50.432851 percent accurate


In [123]:
# Add to df
test_df["event_DT"] = DT_pred
test_df["event_RF"] = RF_pred

LSTM for time

In [125]:
df_result = result_data.sort_values(by=["case", "event"])

df_result["duration"] = pd.to_numeric(pd.to_datetime(df_result["completeTime"]) - pd.to_datetime(df_result["startTime"]))
df_result['REG_DATE'] = pd.to_numeric(pd.to_datetime(df_result['REG_DATE']))
df_result['completeTime'] = pd.to_numeric(pd.to_datetime(df_result['completeTime']))
df_result['startTime'] = pd.to_numeric(pd.to_datetime(df_result['startTime']))

df_filtered = df_result[df_result["case"] < len(df_result)]
df_prediction = df_result[df_result["case"] >= len(df_result)]



df_filtered_new = [v for _, v in df_filtered.groupby('event')]
dictionary_event_startTime = np.empty(len(df_filtered_new), dtype=object) 
for idx, val in enumerate(df_filtered_new):
    dictionary_event_startTime[idx] = val["startTime"]

listVal = df_filtered
listVal


listValSelected = listVal[['case', 'event', 'startTime', 'completeTime', 'AMOUNT_REQ', 'REG_DATE']]
listValSelected_prediction = df_prediction[['case', 'event', 'startTime', 'completeTime', 'AMOUNT_REQ', 'REG_DATE']]
listValSelected_prediction = listValSelected_prediction.values.astype('float32')
listValDuration_prediction = df_prediction['duration']
listValDuration_prediction = listValDuration_prediction.values.astype('float32')
listValDuration = listVal['duration'].astype('float32')

listValSelected = listValSelected.values.astype('float32')

In [128]:
# choose a number of time steps
n_steps = len(listValSelected[0])
# split into samples


n_features = 1
X = listValSelected.reshape((listValSelected.shape[0], listValSelected.shape[1], n_features))
y = listValDuration

# define model
model = Sequential()
model.add(LSTM(50, input_shape=(n_steps, n_features), return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(50, activation='relu'))
# Dropout for regularization
model.add(Dropout(0.5))
# model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=10, verbose=2, workers=-1)

# demonstrate prediction
x_input = listValSelected_prediction[:10]

x_input = x_input.reshape((x_input.shape[0], n_steps, n_features))

Epoch 1/10
2477/2477 - 18s - loss: 20698621133135608274747392.0000 - 18s/epoch - 7ms/step
Epoch 2/10
2477/2477 - 16s - loss: 20698628050664635915829248.0000 - 16s/epoch - 6ms/step
Epoch 3/10
2477/2477 - 17s - loss: 20698607298077552992583680.0000 - 17s/epoch - 7ms/step
Epoch 4/10
2477/2477 - 15s - loss: 20698609603920562206277632.0000 - 15s/epoch - 6ms/step
Epoch 5/10
2477/2477 - 15s - loss: 20698623438978617488441344.0000 - 15s/epoch - 6ms/step
Epoch 6/10
2477/2477 - 15s - loss: 20698614215606580633665536.0000 - 15s/epoch - 6ms/step
Epoch 7/10
2477/2477 - 15s - loss: 20698593463019497710419968.0000 - 15s/epoch - 6ms/step
Epoch 8/10
2477/2477 - 15s - loss: 20698621133135608274747392.0000 - 15s/epoch - 6ms/step
Epoch 9/10
2477/2477 - 15s - loss: 20698598074705516137807872.0000 - 15s/epoch - 6ms/step
Epoch 10/10
2477/2477 - 15s - loss: 20698614215606580633665536.0000 - 15s/epoch - 6ms/step


In [129]:
yhat = model.predict(x_input, verbose=2)

mean_absolute_error(listValDuration_prediction[:50], yhat[0][:50])

1/1 - 0s - 455ms/epoch - 455ms/step


182805170000.0

Neural network for event prediction

In [132]:
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from numba import jit
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from tqdm.keras import TqdmCallback
import seaborn as sns
from sklearn.linear_model import HuberRegressor
import statsmodels.api as sm
from sklearn.linear_model import RANSACRegressor

In [133]:
df = pd.read_csv('export2018.csv')

In [134]:
df = df.sort_values(by=['case','startTime'])
df = df.reset_index()
#A function for determining the true next event for each event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

df['next_event'] = calculator_nb(df['case'].values, df['event'].values)


#A function for determining the true next event for each event
@jit(parallel = True)
def calculator_nb(case, startTime):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = startTime[idx + 1]
        else:
            res[idx] = startTime[idx]

        idx+=1
    return res

df['completeTime'] = calculator_nb(df['case'].values, df['startTime'].values)
df.at[2514265, 'completeTime'] = df.at[2514265, 'startTime']

df['startTime'] =  pd.to_datetime(df['startTime'])
df['completeTime'] =  pd.to_datetime(df['completeTime'])
df['duration'] = df['completeTime'] - df['startTime']

duration = df['duration']
duration = duration / np.timedelta64(1, 's')
df['duration'] = duration

C:\Users\chbak\AppData\Local\Temp/ipykernel_13132/770568818.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13132\770568818.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
numba.core.transforms INFO  finding looplift candidates
C:\Users\chbak\AppData\Local\Temp/ipykernel_13132/770568818.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_13132\770568818.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "ca

In [135]:
train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [136]:
train = train.reset_index()
validate = validate.reset_index()
test = test.reset_index()

In [137]:
ordinal_encoder = OrdinalEncoder()
label_encoder = LabelEncoder()

In [138]:
def prepfeatures_OLS(df_name):
    startTime = pd.to_datetime(df_name['startTime'])
    
    for i in range(len(startTime)):
        startTime[i] = startTime[i].timestamp()
        
    startTime = np.array(startTime).reshape(-1,1)
    
    event = df_name['event'].to_numpy()
    event = event.reshape(-1,1)
    event = ordinal_encoder.fit_transform(event)
    
    #payment_actual0 = normalize(df_name,'payment_actual0')
    #penalty_amount0 = normalize(df_name,'penalty_amount0')
    #number_parcels = normalize(df_name,'number_parcels')
    #area = normalize(df_name,'area')
    payment_actual0 = df_name['payment_actual0'].to_numpy()
    penalty_amount0 = df_name['penalty_amount0'].to_numpy()
    number_parcels = df_name['number_parcels'].to_numpy()
    area = df_name['area'].to_numpy()
    
    X = []
    for i in range(len(event)):
        current = startTime[i]
        current = np.append(current, event[i])
        current = np.append(current, payment_actual0[i])
        current = np.append(current, penalty_amount0[i])
        current = np.append(current, number_parcels[i])
        current = np.append(current, area[i])
        X.append(current)
        
    return np.array(X, dtype=float)

In [139]:
def preplabels_OLS(df_name):
    duration = df_name['duration'].to_numpy()
    return np.array(duration, dtype=float)

In [140]:
X = prepfeatures_OLS(train)
y = preplabels_OLS(train)
huber = HuberRegressor().fit(X, y)
X_test = prepfeatures_OLS(test)
test['predicted'] = huber.predict(X_test)
test['error'] = np.absolute(test['duration'] - test['predicted'])
test['error'].mean()

503611.66394356015

In [141]:
def normalize(df_name, col_name):
    col_as_array = df_name[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.5, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

def prepfeatures(df_name):
    event = df_name['event'].to_numpy()
    event = event.reshape(-1,1)
    event = ordinal_encoder.fit_transform(event)
    
    number_parcels = normalize(df_name,'number_parcels')
    payment_actual0 = normalize(df_name,'payment_actual0')
    area = normalize(df_name,'area')
    cross_compliance = normalize(df_name,'cross_compliance')
    penalty_amount0 = normalize(df_name,'penalty_amount0')
    
    features = []
    for i in range(len(event)):
        current = event[i]
        current = np.append(current,number_parcels[i])
        current = np.append(current,payment_actual0[i])
        current = np.append(current,area[i])
        current = np.append(current,cross_compliance[i])
        current = np.append(current,penalty_amount0[i])
        features.append(current)
        
    return np.array(features)

def preplabels(df_name):
    labels = df_name['next_event'].to_numpy()
    labels = label_encoder.fit_transform(labels)
    labels = labels.reshape(-1, 1)
    
    return np.array(labels)

features = prepfeatures(train)
labels = preplabels(train)

In [142]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(35, activation='relu'),
    keras.layers.Dense(42, activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [143]:
model.fit(features,labels,epochs=5,verbose=1, n_jobs=-1)

Epoch 1/5
47143/47143 [==============================] - 105s 2ms/step - loss: 1.4937 - accuracy: 0.5415
Epoch 2/5
47143/47143 [==============================] - 97s 2ms/step - loss: 1.3633 - accuracy: 0.5703
Epoch 3/5
47143/47143 [==============================] - 115s 2ms/step - loss: 1.3446 - accuracy: 0.5729
Epoch 4/5
47143/47143 [==============================] - 117s 2ms/step - loss: 1.3348 - accuracy: 0.5746
Epoch 5/5
47143/47143 [==============================] - 101s 2ms/step - loss: 1.3278 - accuracy: 0.5763


In [146]:
features_test = prepfeatures(test)
labels_test = preplabels(test)

In [145]:
score = model.evaluate(features_test, labels_test, verbose=1)
print(score)

15715/15715 [==============================] - 22s 1ms/step - loss: 1.7102 - accuracy: 0.4622
[1.7102397680282593, 0.4622116982936859]
